# DF902 딥러닝으로 은하분류하기

## 01. 패키지불러오기

In [1]:
# 패키지 불러오기

import numpy as np
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import Activation
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


- Dense() : 전결합층(fully-conntected layer)을 통과한 출력(output) 레이어
- Conv2D() : 필터로 특징을 뽑아주는 컨볼루션(Convolution) 레이어
- MaxPooling2D() : 사소한 변화를 무시해주는 맥스풀링(Max Pooling) 레이어
- Flatten() : 영상을 일차원으로 바꿔주는 플래튼(Flatten) 레이어"

In [2]:
# 랜덤시드 고정

np.random.seed(3)

In [3]:
# 데이터 생성
# ImageDataGenerator를 쓰는이유
# 사진의 크기 rescale,0~255의 픽셀값을 0~1로 조정 (신경망에서는 작은 입력값을 선호하기 때문)
# RGB픽셀값으로 읽음,부동소수타입의 텐서

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/Users/hyunji/Dafit/DF902_CNN_ClassifyGalaxies/DF902_Data_Galaxy_Resize/train_galaxy',
    target_size=(50, 50),
    batch_size=3,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    '/Users/hyunji/Dafit/DF902_CNN_ClassifyGalaxies/DF902_Data_Galaxy_Resize/val_galaxy',
    target_size=(50, 50),
    batch_size=3,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    '/Users/hyunji/Dafit/DF902_CNN_ClassifyGalaxies/DF902_Data_Galaxy_Resize/test_galaxy',
    target_size=(50, 50),
    batch_size=3,
    class_mode='categorical')

Found 10233 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [4]:
train_generator

In [5]:
# 모델 구성

#from keras.models import Sequential
#from keras.layers import Dense,Activation,Conv2D,MaxPooling2D,Flatten

model = Sequential()
#filters=32, kernel_size=(3,3)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3), strides=2, padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu', strides=2, padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 25, 25, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               73856     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                

- relu: relu()는 활성화함수 중 하나로 입력이 0 이상이면 그대로, 0 이하면 0 출력하는 함수 입니다. 기울기소실의 문제 보완해주고 계산이 빠르나 정보손실의 문제가 발생한다는 단점이 있습니다.

- softmax: softmax()는 활성화함수 중 하나로 출력값을 0~1사이의 값으로 정규화해주는 함수입니다. 또한 출력값들의 총합은 1이 되기 때문에 출력값을 확률로 해석할 수 있다는 성질이 있습니다. 오버피팅의 문제가 발생할 수 있다는 단점이 있습니다.

In [6]:
# 모델 학습과정 설정

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

- loss: 손실 함수는 학습을 통해 얻은 데이터의 추정치가 실제 데이터와 얼마나 차이나는지 평가하는 지표로 쓰입니다. 이 값이 0 에 가까울수록 '무손실' 즉 완벽하게 추정할 수 있다는 의미가 됩니다. 손실함수의 종류는 크게 MSE(Mean of Squared Error)와 크로스 엔트로피가 있으며, 크로스 에트로피는 이른바 원-핫 코딩을 할 때 유효한 수단이 되며, 주로 분류 문제에 사용됩니다.
 
- optimizer: 로컬미니마와 플래튜 문제를 해결하기 위해 사용되는 방법으로 옵티마이저는 학습을 빠르고 안정적으로 하는 최적화방법입니다.Adam은 방향성과 스텝사이즈를 고려하는 옵티마이져로 최근 가장 많이 사용되는 방법입니다.


In [7]:
# 모델 학습시키기

hist=model.fit(train_generator, 
               #steps_per_epoch=3411, 
               epochs=50,
               validation_data=val_generator, 
               validation_steps=4)  

Epoch 1/50
3411/3411 [==============================] - 20s 6ms/step - loss: 0.6739 - accuracy: 0.7141 - val_loss: 0.2220 - val_accuracy: 0.6667
Epoch 2/50
3411/3411 [==============================] - 18s 5ms/step - loss: 0.5457 - accuracy: 0.7816 - val_loss: 1.7627 - val_accuracy: 0.7500
Epoch 3/50
3411/3411 [==============================] - 18s 5ms/step - loss: 0.5090 - accuracy: 0.7946 - val_loss: 1.3532 - val_accuracy: 0.6667
Epoch 4/50
3411/3411 [==============================] - 18s 5ms/step - loss: 0.4705 - accuracy: 0.8139 - val_loss: 0.1402 - val_accuracy: 0.9167
Epoch 5/50
3411/3411 [==============================] - 19s 5ms/step - loss: 0.4484 - accuracy: 0.8212 - val_loss: 0.2944 - val_accuracy: 0.9167
Epoch 6/50
3411/3411 [==============================] - 18s 5ms/step - loss: 0.4270 - accuracy: 0.8302 - val_loss: 0.0506 - val_accuracy: 0.7500
Epoch 7/50
3411/3411 [==============================] - 19s 5ms/step - loss: 0.4032 - accuracy: 0.8364 - val_loss: 0.7645 - val_ac

In [9]:
# 모델 평가

print("--Evaluate--")
scores = model.evaluate_generator(test_generator, steps=4)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

--Evaluate--
accuracy: 83.33%
